In [ ]:
import sklearn
sklearn.__version__

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sea

In [ ]:
import os, types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_2fbe464aba2d4d5f801a3b27d6a0fbc3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='gYhE0tBjj1XMAMutTazvMplL5l1R2lN3h6e9CA2PUJds',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

body = client_2fbe464aba2d4d5f801a3b27d6a0fbc3.get_object(Bucket='carperformanceprediction-donotdelete-pr-nufk2va5q9cuje',Key='car_dataset.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)


In [ ]:
data.drop("car name",axis=1,inplace=True)
data.drop("origin",axis=1,inplace=True)

In [ ]:
data.describe()

In [ ]:
data.isnull().any()

In [ ]:
sea.boxplot(data["mpg"])

In [ ]:
q3=data["mpg"].describe()["75%"]
q1=data["mpg"].describe()["25%"]
iqr=q3-q1
ub=q3+(1.5*iqr)

In [ ]:
data[data["mpg"]>ub]

In [ ]:
(data[data["mpg"]>ub].shape[0] / data["mpg"].shape[0])*100

In [ ]:
outi=data[data["mpg"]>ub].index
data.drop(outi,inplace=True)

In [ ]:
sea.boxplot(data["cylinders"])

In [ ]:
sea.boxplot(data["displacement"])

In [ ]:
sea.boxplot(data["horsepower"])

In [ ]:
q3=data["horsepower"].describe()["75%"]
q1=data["horsepower"].describe()["25%"]
iqr=q3-q1
ub=q3+(1.5*iqr)

In [ ]:
data[data["horsepower"]>ub]

In [ ]:
(data[data["horsepower"]>ub].shape[0] / data["horsepower"].shape[0])*100

#### we dont drop since 2.51

In [ ]:
sea.boxplot(data["weight"])

In [ ]:
sea.boxplot(data["acceleration"])

In [ ]:
q3=data["acceleration"].describe()["75%"]
q1=data["acceleration"].describe()["25%"]
iqr=q3-q1
lb=q1-(1.5*iqr)
ub=q3+(1.5*iqr)

In [ ]:
data[data["acceleration"]<lb]

In [ ]:
data[data["acceleration"]>ub]

In [ ]:
(data[data["acceleration"]>ub].shape[0] / data["acceleration"].shape[0])*100

In [ ]:
(data[data["acceleration"]<lb].shape[0] / data["acceleration"].shape[0])*100

In [ ]:
outi1=data[data["acceleration"]<lb].index
outi2=data[data["acceleration"]>ub].index
data.drop(outi1,inplace=True)
data.drop(outi2,inplace=True)

In [ ]:
sea.boxplot(data["model year"])

#### Model Building

In [ ]:
x=data.iloc[:,1:].values
y=data.iloc[:,0].values

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=73)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
car_model = RandomForestRegressor(n_estimators=30,criterion='mse')
car_model.fit(x_train,y_train)

In [ ]:
y_pred=car_model.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
accuracy = r2_score(y_test,y_pred)

In [ ]:
accuracy

In [ ]:
y_pred_train=car_model.predict(x_train)

In [ ]:
accuracy = r2_score(y_train,y_pred_train)

## deployment

In [ ]:
import ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient
import json

In [ ]:
wml_credentials = {
    "apikey" : "XvwSj9qdy7B_gr4BNjwR4p7-hlUMoZSDWjwHjgqhYZ8s",
    "url" : "https://eu-gb.ml.cloud.ibm.com"
}

In [ ]:
wml_client = APIClient(wml_credentials)
wml_client.spaces.list()

In [ ]:
SPACE_ID = "4339f940-c21a-468a-81df-fbaed55d3c95"

In [ ]:
wml_client.set.default_space(SPACE_ID)

In [ ]:
MODEL_NAME = "car perf pred"
DEPLOYMENT_NAME = "car deployment"
BEST_MODEL = car_model

In [ ]:
software_spec_uid = wml_client.software_specifications.get_id_by_name("default_py3.8")

model_props = {
    wml_client.repository.ModelMetaNames.NAME : MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE : 'scikit-learn_0.23',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : software_spec_uid,
}

In [ ]:
model_details = wml_client.repository.store_model(
    model = BEST_MODEL,
    meta_props = model_props,
    training_data = x_train,
    training_target = y_train,
)

In [ ]:
software_spec_uid

In [ ]:
model_uid = wml_client.repository.get_model_id(model_details)

In [ ]:
model_uid

In [ ]:
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME : DEPLOYMENT_NAME,
    wml_client.deployments.ConfigurationMetaNames.ONLINE : {}
}

In [ ]:
deployment = wml_client.deployments.create(artifact_uid=model_uid , meta_props=deployment_props)

In [ ]:
deployment_uid = wml_client.deployments.get_uid(deployment)

In [ ]:
deployment_uid

In [ ]:
import requests

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
API_KEY = "XvwSj9qdy7B_gr4BNjwR4p7-hlUMoZSDWjwHjgqhYZ8s"
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"input_data": [{"fields": ['cylinders','displacement','horsepower','weight','acceleration','model year'], "values": [[8,440.0,215,4312,8.5,70]]}]}

response_scoring = requests.post('https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/4551b24d-4ed8-457f-b3bc-62dafaefc7b7/predictions?version=2022-03-05', json=payload_scoring, headers={'Authorization': 'Bearer ' + mltoken})
print("Scoring response")
print(response_scoring.json())